In [772]:
import vl_convert as vlc
import json
import copy
import pandas as pd

In [773]:
normalize={'date':'date','Date':'date','day':'date','Day':'date','month':'month','Month':'month','year':'year','Year':'year'}
time_format={'date':'D','Date':'D','day':'D','Day':'D','month':'M','Month':'M','year':'Y','Year':'Y'}
def get_opposite_color(hex_color):
	hex_color = hex_color.lstrip('#')
	r = int(hex_color[0:2], 16)
	g = int(hex_color[2:4], 16)
	b = int(hex_color[4:6], 16)
	r = 255 - r
	g = 255 - g
	b = 255 - b
	r_hex = format(r, '02x')
	g_hex = format(g, '02x')
	b_hex = format(b, '02x')
	return f"#{r_hex}{g_hex}{b_hex}"
def remove_lonely_numbers(lst):
	result = []
	lst=sorted(lst)
	for i in range(len(lst)):
		if (i == 0):
			if (lst[i+1] - lst[i] == 1):  
				result.append(lst[i])
		if (i == len(lst) - 1):
			if(lst[i] - lst[i-1] == 1):
				result.append(lst[i])
		if  (i!=0 and i!=len(lst)-1):
			if (lst[i] - lst[i-1] == 1) | (lst[i+1] - lst[i] == 1):
				result.append(lst[i])
	return result

def generate_conditions(numlist,mainField):
	filtered_years = remove_lonely_numbers(numlist)
	print(filtered_years)
	ranges = []
	start = None
	for year in filtered_years:
		if start is None:
			start = year
		elif year + 1 not in filtered_years:
			end = year
			if start != end:
				ranges.append((start, end))
			start = None
	if start is not None:
		end = filtered_years[-1]
		if start != end:
			ranges.append((start, end))
	conditions = [f"({normalize[mainField]}(datum['{mainField}']) >= {start-1} && {normalize[mainField]}(datum['{mainField}']) <= {end-1})" for start, end in ranges ]
	return conditions

def BarMark(vega,mainField, subField, mainType, subType, value):
	newcolor = get_opposite_color(vega["encoding"]["color"]["value"])

	if mainType == 'temporal':
		# vega["data"]["format"]= {"type": "csv", "parse": {f'{mainField}': f"date:'%{time_format[mainField]}'"}}
		vega["layer"].append( {
		"mark": {"type": "point", "filled": True},
		"transform": [{"filter": f"{normalize[mainField]}(datum['{mainField}']) == {value}"}],
		"encoding": {
			"color": {"value": newcolor},
			"x": {"type": mainType, "field": mainField},
			"y": {"type": subType, "field": subField}
		}})
	else:
		vega["layer"].append( {
			"mark": {"type": "point", "filled": True},
			"transform": [{"filter": f"datum['{mainField}'] == '{value}'"}], 
			"encoding": {
				"color": {"value": newcolor},
				"size": {"value": 100},
				"x": {"type": mainType, "field": mainField, "axis": {}, "bin": False},
				"y": {"type": subType, "field": subField}
			}})
	return vega

def BarText(vega,mainField, subField, mainType, subType, value, text,degree=0):
	newcolor = get_opposite_color(vega["encoding"]["color"]["value"])
	size=14
	if degree!=0:
		size=100
	if mainType == 'temporal':
		vega["data"]["format"]= {"type": "csv", "parse": {f'{mainField}': f"date:'%{time_format[mainField]}'"}}
		vega["layer"].append( {
		"mark": {"type": "text", "filled": True,"fontSize":size,'angle':degree,'dy':-10},
		"transform": [{"filter": f"{normalize[mainField]}(datum['{mainField}']) == {value}"}],
		"encoding": {
			"text" :{"value": text},
			"color": {"value": newcolor},
			"x": {"type": mainType, "field": mainField},
			"y": {"type": subType, "field": subField},
			"fontSize":{"value":size}
		}})
	else:
		vega["layer"].append( {
			"mark": {"type": "text", "filled": True,"fontsize":size,'angle':degree,'dy':-10},
			"transform": [{"filter": f"datum['{mainField}'] == '{value}'"}], # type: ignore
			"encoding": {
				"text" :{"value": text},
				"color": {"value": newcolor},
				"x": {"type": mainType, "field": mainField},
				"y": {"type": subType, "field": subField},
				"fontSize":{"value":size}
			}})
	return vega
def BarLine(vega,mainField, subField, mainType, subType, value,value1=0,subValue=0,subValue1=0):
	newcolor = get_opposite_color(vega["encoding"]["color"]["value"])
	if subValue==0:
		if vega["encoding"]["x"]["field"] == subField:
			vega["layer"].append({ "data": { "values": [{}] }, "mark": { "type": "rule", "color": newcolor }, "encoding": { "x": { "datum": value } } })
			if value1!=0:
				vega["layer"].append({ "data": { "values": [{}] }, "mark": { "type": "rule", "color": newcolor }, "encoding": { "x": { "datum": value1 } } })
		else:
			vega["layer"].append({ "data": { "values": [{}] }, "mark": { "type": "rule", "color": newcolor }, "encoding": { "y": { "datum": value } } })
			if value1!=0:
				vega["layer"].append({ "data": { "values": [{}] }, "mark": { "type": "rule", "color": newcolor }, "encoding": { "y": { "datum": value1 } } })
	else:
		vega["layer"].append({
      "data": {
        "values": [
          {"PlotX1": value, "PlotY1": value1, "PlotX2": subValue, "PlotY2": subValue1}
        ]
      },
      "mark": {
        "type": "rule",
        "stroke": newcolor,
        "strokeWidth": 2
      },
      "encoding": {
        "x": {"field": "PlotX1", "type": mainType},
        "y": {"field": "PlotY1", "type": subType},
        "x2": {"field": "PlotX2", "type": mainType},
        "y2": {"field": "PlotY2", "type": subType}
      }
    })

	return vega
def BarColor(vega,mainField, subField, mainType, subType, value, condition,value1=0):
	newcolor = get_opposite_color(vega["encoding"]["color"]["value"])
	new_layer = copy.deepcopy(vega["layer"][0])
	if condition == 'equal':
		new_layer["transform"] = [{"filter": f"datum['{subField}'] == {value}"}]
	elif condition == 'above':
		new_layer["transform"] = [{"filter": f"datum['{subField}'] >= {value}"}]
	elif condition == 'below':
		new_layer["transform"] = [{"filter": f"datum['{subField}'] <= {value}"}]
	elif condition == 'above_and_below':
		new_layer["transform"] = [{"filter": f"datum['{subField}'] >= {value} && datum['{subField}'] <= {value1}"}]
	elif condition == 'left_bound':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] >= {value}"}]
	elif condition == 'right_bound':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] <= {value}"}]
	elif condition == 'left_and_right':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] >= {value} && datum['{mainField}'] <= {value1}"}]
	elif condition == 'compare':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] == '{value}' || datum['{mainField}'] == '{value1}'"}]

	new_layer["encoding"]["color"]["value"] = newcolor
	vega["layer"].append(new_layer)
	return vega
def BarOpacity(vega,mainField, subField, mainType, subType, value,condition,value1=0):
	vega["layer"][0]["encoding"]["opacity"]={"value": 0.4}
	new_layer = copy.deepcopy(vega["layer"][0])
	if condition == 'equal':
		new_layer["transform"] = [{"filter": f"datum['{subField}'] == {value}"}]
	elif condition == 'above':
		new_layer["transform"] = [{"filter": f"datum['{subField}'] >= {value}"}]
	elif condition == 'below':
		new_layer["transform"] = [{"filter": f"datum['{subField}'] <= {value}"}]
	elif condition == 'above_and_below':
		new_layer["transform"] = [{"filter": f"datum['{subField}'] >= {value} && datum['{subField}'] <= {value1}"}]
	elif condition == 'left_bound':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] >= {value}"}]
	elif condition == 'right_bound':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] <= {value}"}]
	elif condition == 'left_and_right':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] >= {value} && datum['{mainField}'] <= {value1}"}]
	elif condition == 'compare':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] == '{value}' || datum['{mainField}'] == '{value1}'"}]
				
	new_layer["encoding"]["opacity"]={"value": 1}
	vega["layer"].append(new_layer)
	return vega

def BarArea(vega,mainField, subField, mainType, subType, value,value1=0,subValue=0,subValue1=0,shade=True):
    newcolor = get_opposite_color(vega["encoding"]["color"]["value"])
    # df = pd.read_csv(vega["data"]["url"])
    if shade==True:
        stroke=0
        opacity=0.2
    else:
        stroke=2
        opacity=0

    vega["layer"].append({
      "data": {
        "values": [
          {"PlotX1": value, "PlotY1": value1, "PlotX2": subValue, "PlotY2": subValue1}
        ]
      },
        "mark": {"type": "rect", "color": newcolor,"stroke":newcolor,"fillOpacity":opacity,"strokeWidth":stroke},
      "encoding": {
        "x": {"field": "PlotX1", "type": mainType},
        "y": {"field": "PlotY1", "type": subType},
        "x2": {"field": "PlotX2", "type": mainType},
        "y2": {"field": "PlotY2", "type": subType}
    }})
    return vega

In [774]:
vl_spec=r"""
{
  "config": {
    "background": "#fff",
    "area": {"fill": "#3e5c69"},
    "line": {"stroke": "#3e5c69"},
    "rect": {"fill": "#3e5c69"},
    "bar": {"fill": "#3e5c69"},
    "point": {"color": "#3e5c69"},
    "axis": {
      "domainWidth": 0.5,
      "grid": true,
      "labelPadding": 2,
      "tickSize": 5,
      "tickWidth": 0.5,
      "titleFontWeight": "normal"
    },
    "axisBand": {"grid": false},
    "axisX": {"gridWidth": 0.2},
    "axisY": {"gridDash": [3], "gridWidth": 0.4},
    "legend": {"labelFontSize": 11, "padding": 1, "symbolType": "square"},
    "range": {
      "category": [
        "#3e5c69",
        "#6793a6",
        "#182429",
        "#0570b0",
        "#3690c0",
        "#74a9cf",
        "#a6bddb",
        "#e2ddf2"
      ]
    }
  },
  "data": {
    "url": "https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/1048.csv"
  },
  "mark": "bar",
  "encoding": {
    "color": {"value": "#0570b0"},
    "x": {"type": "nominal", "axis": {}, "bin": false, "field": "Year"},
    "y": {
      "type": "quantitative",
      "axis": {"title": "National debt in billion U.S. dollars"},
      "field": "National debt in billion U\\.S\\. dollars"
    }
  },
  "title": [
    "Ireland : National debt from 2014 to 2024",
    "(in billion U.S. dollars)"
  ],
  "$schema": "https://vega.github.io/schema/vega-lite/v4.8.1.json"
}
"""

In [775]:
png_data = vlc.vegalite_to_png(vl_spec=vl_spec, scale=2)
with open("Sample-Bar-RetrieveValue.png", "wb") as f:
	f.write(png_data)

In [776]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarMark(vl_json, "Year", "National debt in billion U\\.S\\. dollars", "nominal", "quantitative", 2016)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("RetrieveValue-Bar-Mark.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#3e5c69'}, 'line': {'stroke': '#3e5c69'}, 'rect': {'fill': '#3e5c69'}, 'bar': {'fill': '#3e5c69'}, 'point': {'color': '#3e5c69'}, 'axis': {'domainWidth': 0.5, 'grid': True, 'labelPadding': 2, 'tickSize': 5, 'tickWidth': 0.5, 'titleFontWeight': 'normal'}, 'axisBand': {'grid': False}, 'axisX': {'gridWidth': 0.2}, 'axisY': {'gridDash': [3], 'gridWidth': 0.4}, 'legend': {'labelFontSize': 11, 'padding': 1, 'symbolType': 'square'}, 'range': {'category': ['#3e5c69', '#6793a6', '#182429', '#0570b0', '#3690c0', '#74a9cf', '#a6bddb', '#e2ddf2']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/1048.csv'}, 'title': ['Ireland : National debt from 2014 to 2024', '(in billion U.S. dollars)'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#0570b0'}, 'x': {'type': 'nominal', 'axis': {}, 'bin': False, 'field': 'Ye

In [777]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarText(vl_json, "Year", "National debt in billion U\\.S\\. dollars", "nominal", "quantitative", 2016,"lowest")
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("RetrieveValue-Bar-Text.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#3e5c69'}, 'line': {'stroke': '#3e5c69'}, 'rect': {'fill': '#3e5c69'}, 'bar': {'fill': '#3e5c69'}, 'point': {'color': '#3e5c69'}, 'axis': {'domainWidth': 0.5, 'grid': True, 'labelPadding': 2, 'tickSize': 5, 'tickWidth': 0.5, 'titleFontWeight': 'normal'}, 'axisBand': {'grid': False}, 'axisX': {'gridWidth': 0.2}, 'axisY': {'gridDash': [3], 'gridWidth': 0.4}, 'legend': {'labelFontSize': 11, 'padding': 1, 'symbolType': 'square'}, 'range': {'category': ['#3e5c69', '#6793a6', '#182429', '#0570b0', '#3690c0', '#74a9cf', '#a6bddb', '#e2ddf2']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/1048.csv'}, 'title': ['Ireland : National debt from 2014 to 2024', '(in billion U.S. dollars)'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#0570b0'}, 'x': {'type': 'nominal', 'axis': {}, 'bin': False, 'field': 'Ye

In [778]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarLine(vl_json, "Year", "National debt in billion U\\.S\\. dollars", "nominal", "quantitative", 220.87)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("RetrieveValue-Bar-Line.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#3e5c69'}, 'line': {'stroke': '#3e5c69'}, 'rect': {'fill': '#3e5c69'}, 'bar': {'fill': '#3e5c69'}, 'point': {'color': '#3e5c69'}, 'axis': {'domainWidth': 0.5, 'grid': True, 'labelPadding': 2, 'tickSize': 5, 'tickWidth': 0.5, 'titleFontWeight': 'normal'}, 'axisBand': {'grid': False}, 'axisX': {'gridWidth': 0.2}, 'axisY': {'gridDash': [3], 'gridWidth': 0.4}, 'legend': {'labelFontSize': 11, 'padding': 1, 'symbolType': 'square'}, 'range': {'category': ['#3e5c69', '#6793a6', '#182429', '#0570b0', '#3690c0', '#74a9cf', '#a6bddb', '#e2ddf2']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/1048.csv'}, 'title': ['Ireland : National debt from 2014 to 2024', '(in billion U.S. dollars)'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#0570b0'}, 'x': {'type': 'nominal', 'axis': {}, 'bin': False, 'field': 'Ye

In [779]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarColor(vl_json,"Year", "National debt in billion U\\.S\\. dollars", "nominal", "quantitative", 220.87,'equal')
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("RetrieveValue-Bar-Color.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#3e5c69'}, 'line': {'stroke': '#3e5c69'}, 'rect': {'fill': '#3e5c69'}, 'bar': {'fill': '#3e5c69'}, 'point': {'color': '#3e5c69'}, 'axis': {'domainWidth': 0.5, 'grid': True, 'labelPadding': 2, 'tickSize': 5, 'tickWidth': 0.5, 'titleFontWeight': 'normal'}, 'axisBand': {'grid': False}, 'axisX': {'gridWidth': 0.2}, 'axisY': {'gridDash': [3], 'gridWidth': 0.4}, 'legend': {'labelFontSize': 11, 'padding': 1, 'symbolType': 'square'}, 'range': {'category': ['#3e5c69', '#6793a6', '#182429', '#0570b0', '#3690c0', '#74a9cf', '#a6bddb', '#e2ddf2']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/1048.csv'}, 'title': ['Ireland : National debt from 2014 to 2024', '(in billion U.S. dollars)'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#0570b0'}, 'x': {'type': 'nominal', 'axis': {}, 'bin': False, 'field': 'Ye

In [780]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarOpacity(vl_json,"Year", "National debt in billion U\\.S\\. dollars", "nominal", "quantitative", 220.87,'equal')
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("RetrieveValue-Bar-Opacity.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#3e5c69'}, 'line': {'stroke': '#3e5c69'}, 'rect': {'fill': '#3e5c69'}, 'bar': {'fill': '#3e5c69'}, 'point': {'color': '#3e5c69'}, 'axis': {'domainWidth': 0.5, 'grid': True, 'labelPadding': 2, 'tickSize': 5, 'tickWidth': 0.5, 'titleFontWeight': 'normal'}, 'axisBand': {'grid': False}, 'axisX': {'gridWidth': 0.2}, 'axisY': {'gridDash': [3], 'gridWidth': 0.4}, 'legend': {'labelFontSize': 11, 'padding': 1, 'symbolType': 'square'}, 'range': {'category': ['#3e5c69', '#6793a6', '#182429', '#0570b0', '#3690c0', '#74a9cf', '#a6bddb', '#e2ddf2']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/1048.csv'}, 'title': ['Ireland : National debt from 2014 to 2024', '(in billion U.S. dollars)'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#0570b0'}, 'x': {'type': 'nominal', 'axis': {}, 'bin': False, 'field': 'Ye

In [781]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarOpacity(vl_json,"Year", "National debt in billion U\\.S\\. dollars", "nominal", "quantitative", 220.87,'equal')
vl_json=BarText(vl_json, "Year", "National debt in billion U\\.S\\. dollars", "nominal", "quantitative", 2016,"lowest")
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("RetrieveValue-Bar-OpacityText.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#3e5c69'}, 'line': {'stroke': '#3e5c69'}, 'rect': {'fill': '#3e5c69'}, 'bar': {'fill': '#3e5c69'}, 'point': {'color': '#3e5c69'}, 'axis': {'domainWidth': 0.5, 'grid': True, 'labelPadding': 2, 'tickSize': 5, 'tickWidth': 0.5, 'titleFontWeight': 'normal'}, 'axisBand': {'grid': False}, 'axisX': {'gridWidth': 0.2}, 'axisY': {'gridDash': [3], 'gridWidth': 0.4}, 'legend': {'labelFontSize': 11, 'padding': 1, 'symbolType': 'square'}, 'range': {'category': ['#3e5c69', '#6793a6', '#182429', '#0570b0', '#3690c0', '#74a9cf', '#a6bddb', '#e2ddf2']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/1048.csv'}, 'title': ['Ireland : National debt from 2014 to 2024', '(in billion U.S. dollars)'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#0570b0'}, 'x': {'type': 'nominal', 'axis': {}, 'bin': False, 'field': 'Ye

In [782]:
vl_spec=r"""
{
  "config": {
    "background": "#fff",
    "area": {"fill": "#4572a7"},
    "line": {"stroke": "#4572a7", "strokeWidth": 2},
    "rect": {"fill": "#4572a7"},
    "bar": {"fill": "#4572a7"},
    "point": {"color": "#4572a7", "strokeWidth": 1.5, "size": 50},
    "axis": {
      "bandPosition": 0.5,
      "gridColor": "#000000",
      "gridOpacity": 1,
      "gridWidth": 0.5,
      "labelPadding": 10,
      "tickSize": 5,
      "tickWidth": 0.5
    },
    "axisBand": {},
    "legend": {
      "labelBaseline": "middle",
      "labelFontSize": 11,
      "symbolSize": 50,
      "symbolType": "square"
    },
    "range": {
      "category": [
        "#4572a7",
        "#aa4643",
        "#8aa453",
        "#71598e",
        "#4598ae",
        "#d98445",
        "#94aace",
        "#d09393",
        "#b9cc98",
        "#a99cbc"
      ]
    }
  },
  "data": {
    "url": "https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10139.csv"
  },
  "mark": "bar",
  "encoding": {
    "color": {"value": "#d98445"},
    "x": {"type": "nominal", "axis": {"labelAngle": -30}, "field": "Year"},
    "y": {
      "type": "quantitative",
      "axis": {"labelAngle": 30, "title": "Mobile customers in millions"},
      "field": "Mobile customers in millions"
    }
  },
  "title": [
    "Mobile customers in millions in the UK from 2008/09 to 2020/21",
    "(in millions of customers)"
  ],
  "$schema": "https://vega.github.io/schema/vega-lite/v4.8.1.json"
}
"""

In [783]:
png_data = vlc.vegalite_to_png(vl_spec=vl_spec, scale=2)
with open("Sample-Bar-Filter.png", "wb") as f:
	f.write(png_data)

In [784]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarColor(vl_json,"Year", "Mobile customers in millions", "nominal", "quantitative", 30,'below')
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Filter-Bar-Color.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10139.csv'}, 'title': ['Mobile customers in millions in the UK from 2008/09 to 2020/21', '(in millions of customers)'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#d98445'}, 'x': {

In [785]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarLine(vl_json, "Year", "Mobile customers in millions", "nominal", "quantitative", 30)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Filter-Bar-Line.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10139.csv'}, 'title': ['Mobile customers in millions in the UK from 2008/09 to 2020/21', '(in millions of customers)'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#d98445'}, 'x': {

In [786]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarOpacity(vl_json,"Year", "Mobile customers in millions", "nominal", "quantitative", 30,'below')
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Filter-Bar-Opacity.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10139.csv'}, 'title': ['Mobile customers in millions in the UK from 2008/09 to 2020/21', '(in millions of customers)'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#d98445'}, 'x': {

In [787]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarArea(vl_json,"Year", "Mobile customers in millions", "nominal", "quantitative", "2008",0,"2020",30)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Filter-Bar-Shade.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10139.csv'}, 'title': ['Mobile customers in millions in the UK from 2008/09 to 2020/21', '(in millions of customers)'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#d98445'}, 'x': {

In [788]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarOpacity(vl_json,"Year", "Mobile customers in millions", "nominal", "quantitative", 30,'below')
vl_json=BarLine(vl_json, "Year", "Mobile customers in millions", "nominal", "quantitative", 30)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Filter-Bar-OpacityLine.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10139.csv'}, 'title': ['Mobile customers in millions in the UK from 2008/09 to 2020/21', '(in millions of customers)'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#d98445'}, 'x': {

In [789]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarArea(vl_json,"Year", "National debt in billion U\\.S\\. dollars", "nominal", "quantitative", "2008",0,"2020",30,shade=False)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Filter-Bar-Bound.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10139.csv'}, 'title': ['Mobile customers in millions in the UK from 2008/09 to 2020/21', '(in millions of customers)'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#d98445'}, 'x': {

In [790]:
vl_spec=r'''
{
  "config": {
    "background": "#fff",
    "area": {"fill": "#4572a7"},
    "line": {"stroke": "#4572a7", "strokeWidth": 2},
    "rect": {"fill": "#4572a7"},
    "bar": {"fill": "#4572a7"},
    "point": {"color": "#4572a7", "strokeWidth": 1.5, "size": 50},
    "axis": {
      "bandPosition": 0.5,
      "gridColor": "#000000",
      "gridOpacity": 1,
      "gridWidth": 0.5,
      "labelPadding": 10,
      "tickSize": 5,
      "tickWidth": 0.5
    },
    "axisBand": {},
    "legend": {
      "labelBaseline": "middle",
      "labelFontSize": 11,
      "symbolSize": 50,
      "symbolType": "square"
    },
    "range": {
      "category": [
        "#4572a7",
        "#aa4643",
        "#8aa453",
        "#71598e",
        "#4598ae",
        "#d98445",
        "#94aace",
        "#d09393",
        "#b9cc98",
        "#a99cbc"
      ]
    }
  },
  "data": {
    "url": "https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10735.csv"
  },
  "mark": "bar",
  "encoding": {
    "color": {"value": "#d98445"},
    "x": {"type": "nominal", "axis": {"labelAngle": -30}, "field": "Country"},
    "y": {
      "type": "quantitative",
      "axis": {
        "labelAngle": 30,
        "title": "Average daily usage time in minutes"
      },
      "field": "Average daily usage time in minutes"
    }
  },
  "title": ["Average daily usage time of", "smartphones in 2018, by country"],
  "$schema": "https://vega.github.io/schema/vega-lite/v4.8.1.json"
}
'''


In [791]:
png_data = vlc.vegalite_to_png(vl_spec=vl_spec, scale=2)
with open("Sample-Bar-Range.png", "wb") as f:
	f.write(png_data)

In [792]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarColor(vl_json,"Country", "Average daily usage time in minutes", "nominal", "quantitative", 50,'above_and_below',70)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Range-Bar-Color.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10735.csv'}, 'title': ['Average daily usage time of', 'smartphones in 2018, by country'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#d98445'}, 'x': {'type': 'nominal', 'axis': {'l

In [793]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarOpacity(vl_json,"Country", "Average daily usage time in minutes", "nominal", "quantitative", 50,'above_and_below',70)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Range-Bar-Opacity.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10735.csv'}, 'title': ['Average daily usage time of', 'smartphones in 2018, by country'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#d98445'}, 'x': {'type': 'nominal', 'axis': {'l

In [794]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarLine(vl_json, "Country", "Average daily usage time in minutes", "nominal", "quantitative", 50,70)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Range-Bar-Line.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10735.csv'}, 'title': ['Average daily usage time of', 'smartphones in 2018, by country'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#d98445'}, 'x': {'type': 'nominal', 'axis': {'l

In [795]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarOpacity(vl_json,"Country", "Average daily usage time in minutes", "nominal", "quantitative", 50,'above_and_below',70)
vl_json=BarLine(vl_json, "Country", "Average daily usage time in minutes", "nominal", "quantitative", 50,70)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Range-Bar-OpacityLine.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10735.csv'}, 'title': ['Average daily usage time of', 'smartphones in 2018, by country'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#d98445'}, 'x': {'type': 'nominal', 'axis': {'l

In [796]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarArea(vl_json,"Country", "Average daily usage time in minutes", "nominal", "quantitative", "China",50,"Vietnam",70)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Range-Bar-Shade.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10735.csv'}, 'title': ['Average daily usage time of', 'smartphones in 2018, by country'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#d98445'}, 'x': {'type': 'nominal', 'axis': {'l

In [797]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarArea(vl_json,"Country", "Average daily usage time in minutes", "nominal", "quantitative", "China",50,"Vietnam",70,shade=False)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Range-Bar-Bound.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10735.csv'}, 'title': ['Average daily usage time of', 'smartphones in 2018, by country'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#d98445'}, 'x': {'type': 'nominal', 'axis': {'l

In [798]:
vl_spec=r"""

  {
  "config": {
    "background": "#fff",
    "area": {"fill": "#4572a7"},
    "line": {"stroke": "#4572a7", "strokeWidth": 2},
    "rect": {"fill": "#4572a7"},
    "bar": {"fill": "#4572a7"},
    "point": {"color": "#4572a7", "strokeWidth": 1.5, "size": 50},
    "axis": {
      "bandPosition": 0.5,
      "gridColor": "#000000",
      "gridOpacity": 1,
      "gridWidth": 0.5,
      "labelPadding": 10,
      "tickSize": 5,
      "tickWidth": 0.5
    },
    "axisBand": {},
    "legend": {
      "labelBaseline": "middle",
      "labelFontSize": 11,
      "symbolSize": 50,
      "symbolType": "square"
    },
    "range": {
      "category": [
        "#4572a7",
        "#aa4643",
        "#8aa453",
        "#71598e",
        "#4598ae",
        "#d98445",
        "#94aace",
        "#d09393",
        "#b9cc98",
        "#a99cbc"
      ]
    }
  },
  "data": {
    "url": "https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10269.csv"
  },
  "mark": "bar",
  "encoding": {
    "color": {"value": "#d98445"},
    "x": {"type": "nominal", "axis": {"labelAngle": -30}, "field": "Year"},
    "y": {
      "type": "quantitative",
      "axis": {"labelAngle": 30, "title": "Number of employees"},
      "field": "Number of employees"
    }
  },
  "title": [
    "Number of employees at Brunswick",
    "investment banking in the United States from",
    "2009 to 2019"
  ],
  "$schema": "https://vega.github.io/schema/vega-lite/v4.8.1.json"
}
"""

In [799]:
png_data = vlc.vegalite_to_png(vl_spec=vl_spec, scale=2)
with open("Sample-Bar-Trend.png", "wb") as f:
	f.write(png_data)

In [800]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarColor(vl_json,"Year", "Number of employees", "nominal", "quantitative", 2014,'left_and_right',2017)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Trend-Bar-Color.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10269.csv'}, 'title': ['Number of employees at Brunswick', 'investment banking in the United States from', '2009 to 2019'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#d98445'}, 'x

In [801]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarOpacity(vl_json,"Year", "Number of employees", "nominal", "quantitative", 2014,'left_and_right',2017)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Trend-Bar-Opacity.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10269.csv'}, 'title': ['Number of employees at Brunswick', 'investment banking in the United States from', '2009 to 2019'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#d98445'}, 'x

In [802]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarText(vl_json,"Year", "Number of employees", "nominal", "quantitative", 2015,"→",-45)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Trend-Bar-Mark.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10269.csv'}, 'title': ['Number of employees at Brunswick', 'investment banking in the United States from', '2009 to 2019'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#d98445'}, 'x

In [803]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarText(vl_json,"Year", "Number of employees", "nominal", "quantitative", 2015,"increase")
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Trend-Bar-Text.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10269.csv'}, 'title': ['Number of employees at Brunswick', 'investment banking in the United States from', '2009 to 2019'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#d98445'}, 'x

In [804]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarLine(vl_json,"Year", "Number of employees", "nominal", "quantitative", "2014",12000,"2017",15000)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Trend-Bar-Line.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10269.csv'}, 'title': ['Number of employees at Brunswick', 'investment banking in the United States from', '2009 to 2019'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#d98445'}, 'x

In [805]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarArea(vl_json,"Year", "Number of employees", "nominal", "quantitative", "2014",12000,"2017",15000)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Trend-Bar-Shade.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10269.csv'}, 'title': ['Number of employees at Brunswick', 'investment banking in the United States from', '2009 to 2019'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#d98445'}, 'x

In [806]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarArea(vl_json,"Year", "Number of employees", "nominal", "quantitative", "2014",12000,"2017",15000,shade=False)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Trend-Bar-Bound.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10269.csv'}, 'title': ['Number of employees at Brunswick', 'investment banking in the United States from', '2009 to 2019'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#d98445'}, 'x

In [807]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarOpacity(vl_json,"Year", "Number of employees", "nominal", "quantitative", 2014,'left_and_right',2017)
vl_json=BarText(vl_json,"Year", "Number of employees", "nominal", "quantitative", 2015,"→",-45)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Trend-Bar-OpacityMark.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10269.csv'}, 'title': ['Number of employees at Brunswick', 'investment banking in the United States from', '2009 to 2019'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#d98445'}, 'x

In [808]:
vl_spec=r''' 
{
  "config": {
    "background": "#fff",
    "area": {"fill": "#4572a7"},
    "line": {"stroke": "#4572a7", "strokeWidth": 2},
    "rect": {"fill": "#4572a7"},
    "bar": {"fill": "#4572a7"},
    "point": {"color": "#4572a7", "strokeWidth": 1.5, "size": 50},
    "axis": {
      "bandPosition": 0.5,
      "gridColor": "#000000",
      "gridOpacity": 1,
      "gridWidth": 0.5,
      "labelPadding": 10,
      "tickSize": 5,
      "tickWidth": 0.5
    },
    "axisBand": {},
    "legend": {
      "labelBaseline": "middle",
      "labelFontSize": 11,
      "symbolSize": 50,
      "symbolType": "square"
    },
    "range": {
      "category": [
        "#4572a7",
        "#aa4643",
        "#8aa453",
        "#71598e",
        "#4598ae",
        "#d98445",
        "#94aace",
        "#d09393",
        "#b9cc98",
        "#a99cbc"
      ]
    }
  },
  "data": {
    "url": "https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/1351.csv"
  },
  "mark": "bar",
  "encoding": {
    "color": {"value": "#d98445"},
    "x": {"type": "nominal", "axis": {"labelAngle": -30}, "field": "Country"},
    "y": {
      "type": "quantitative",
      "axis": {"labelAngle": 30, "title": "Consumption in exajoules"},
      "field": "Consumption in exajoules"
    }
  },
  "title": ["Leading energy consumers in 2019", "(in exajoules)"],
  "$schema": "https://vega.github.io/schema/vega-lite/v4.8.1.json"
}
'''

In [809]:
png_data = vlc.vegalite_to_png(vl_spec=vl_spec, scale=2)
with open("Sample-Bar-Compare.png", "wb") as f:
	f.write(png_data)

In [810]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarColor(vl_json,"Country", "Consumption in exajoules", "nominal", "quantitative", "U.S.",'compare',"China")
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Compare-Bar-Color.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/1351.csv'}, 'title': ['Leading energy consumers in 2019', '(in exajoules)'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#d98445'}, 'x': {'type': 'nominal', 'axis': {'labelAngle': -

In [811]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarOpacity(vl_json,"Country", "Consumption in exajoules", "nominal", "quantitative", "U.S.",'compare',"China")
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Compare-Bar-Opacity.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/1351.csv'}, 'title': ['Leading energy consumers in 2019', '(in exajoules)'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#d98445'}, 'x': {'type': 'nominal', 'axis': {'labelAngle': -

In [812]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarMark(vl_json,"Country", "Consumption in exajoules", "nominal", "quantitative", "U.S.")
vl_json=BarMark(vl_json,"Country", "Consumption in exajoules", "nominal", "quantitative", "China")
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Compare-Bar-Mark.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/1351.csv'}, 'title': ['Leading energy consumers in 2019', '(in exajoules)'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#d98445'}, 'x': {'type': 'nominal', 'axis': {'labelAngle': -

In [813]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarText(vl_json,"Country", "Consumption in exajoules", "nominal", "quantitative", "U.S.",'lower')
vl_json=BarText(vl_json,"Country", "Consumption in exajoules", "nominal", "quantitative", "China",'higher')
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Compare-Bar-Text.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/1351.csv'}, 'title': ['Leading energy consumers in 2019', '(in exajoules)'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#d98445'}, 'x': {'type': 'nominal', 'axis': {'labelAngle': -

In [814]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=BarOpacity(vl_json,"Year", "National debt in billion U\\.S\\. dollars", "nominal", "quantitative", 2014,'compare',2018)
vl_json=BarText(vl_json,"Country", "Consumption in exajoules", "nominal", "quantitative", "U.S.",'lower')
vl_json=BarText(vl_json,"Country", "Consumption in exajoules", "nominal", "quantitative", "China",'higher')
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Compare-Bar-OpacityText.png", "wb") as f:
	f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/1351.csv'}, 'title': ['Leading energy consumers in 2019', '(in exajoules)'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'bar', 'encoding': {'color': {'value': '#d98445'}, 'x': {'type': 'nominal', 'axis': {'labelAngle': -